In [37]:
import os, random, sys, time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from math import pi

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import NoSuchElementException

import time 
from operator import length_hint 

browser = webdriver.Chrome('/Users/vpaskannaya/Talent Analytics Live/chromedriver')

browser.get('https://www.linkedin.com/uas/login')

file = open('Config.txt')
lines = file.readlines()
username = lines[0]
password = lines[1]

elementID = browser.find_element_by_id('username')
elementID.send_keys(username)

elementID = browser.find_element_by_id('password')
elementID.send_keys(password)

elementID.submit()

visitingcompany = '/search/results/people/?currentCompany=%5B"33223"%5D&'
fulllink = 'https://www.linkedin.com' + visitingcompany
browser.get(fulllink)

In [38]:
#how many profiles can we expect in our list?
results = BeautifulSoup(browser.page_source).find('div',{'class':'search-results-container'}).find('div').get_text(strip=True)
expectedresult=int(results[:results.rfind(' ')])
expectedresult

652

In [39]:
#let's define some helpful functions

#step 1
#we get 10 HTML containers that contain profile links
#how do we know we have 10 containers? we can call len(profilecontainers)


def getcontainers(soup):
    containersqueued = []
    ProfilesID = []
    pav = soup.find('div', {'class':'ph0 pv2 artdeco-card mb2'})
    profilecontainers= pav.findAll('span', {'class':'entity-result__title-text'})
    return profilecontainers

#step 2
#we clean up those 10 HTML containers and extract unique profile link
#the loop below allows to go through each container and grab a link which starts with 'href'

def parsethepage(profiles):
    profilesonpage = []
    i=0
    while i < len(profiles):
        profilesonpage.append(profiles[i].findAll('a',{'class':'app-aware-link'})[0].get('href'))
        i += 1
    return profilesonpage

#vois-la! now, let's do it until the pages run out....

In [40]:
# This code allows scraping all linkedIn profiles as it is flipping to the next page
x = 0
profilesqueued = []

#EDIT NUMBER OF PAGES if needed
numberofpages = 66

while x < numberofpages:
    time.sleep(3)
    this_page = parsethepage(getcontainers(BeautifulSoup(browser.page_source)))
    i=0
    while i < len(this_page):
        profilesqueued.append(this_page[i])
        if len(profilesqueued)==expectedresult:
            break
        i += 1
    browser.execute_script("window.scrollTo(0, 10000);")
    time.sleep(3)
    try:
        nextpage = browser.find_element_by_xpath("//button[@class='artdeco-pagination__button artdeco-pagination__button--next artdeco-button artdeco-button--muted artdeco-button--icon-right artdeco-button--1 artdeco-button--tertiary ember-view']").click()
    except:
        continue
    x += 1

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//button[@class='artdeco-pagination__button artdeco-pagination__button--next artdeco-button artdeco-button--muted artdeco-button--icon-right artdeco-button--1 artdeco-button--tertiary ember-view']"}
  (Session info: chrome=91.0.4472.106)


In [41]:
#export the list into an excel file
import pandas as pd
import numpy as np

linkedin_links = pd.DataFrame(profilesqueued,columns={'url'})

linkedin_links.to_excel('/Users/vpaskannaya/Talent Analytics Live/linkedin_links.xlsx', index = False, header=False)

linkedin_links.tail()

,url
647,https://www.linkedin.com/in/normand-bessette-1...
648,https://www.linkedin.com/in/jazzbhath?miniProf...
649,https://www.linkedin.com/in/fabrice-riva-60bb4...
650,https://www.linkedin.com/in/vbusnita?miniProfi...
651,https://www.linkedin.com/in/victoriapaskannaya...


In [42]:
columns=['url', 'profile_name','job_title','duration','company1','company2']
df = pd.DataFrame(columns=columns)

In [43]:
#this is a new code part
import os, random, sys, time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from math import pi

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

from selenium.common.exceptions import NoSuchElementException

#from linkedin_scraper import Person, actions
from parsel import Selector

browser = webdriver.Chrome('/Users/vpaskannaya/Talent Analytics Live/chromedriver')

browser.get('https://www.linkedin.com/uas/login')

file = open('Config.txt')
lines = file.readlines()
username = lines[0]
password = lines[1]

elementID = browser.find_element_by_id('username')
elementID.send_keys(username)

elementID = browser.find_element_by_id('password')
elementID.send_keys(password)

elementID.submit()

n=0
while n < len(table):
    profilelink = table['url'][n]
    browser.get(profilelink)
    
    time.sleep(2)
    
    page=BeautifulSoup(browser.page_source)

    profile_name=page.find('h1',{'class':'text-heading-xlarge inline t-24 v-align-middle break-words'}).get_text(strip=True)

    job_title = page.find('div',{'class':'text-body-medium break-words'}).get_text(strip=True)
    
    try:
        duration = page.find('span',{'class':'pv-entity__bullet-item-v2'}).get_text(strip=True)
    except:
        duration = None

    previous_employers = page.find('section',{'id':'experience-section'})
    
    time.sleep(2)
    try:
        company1=previous_employers.findAll('p',{'class':'pv-entity__secondary-title'})[1].get_text('|',strip=True)
        company1=company1[:company1.rfind('|')]
    except:
        company1=None

    try:
        company2=previous_employers.findAll('p',{'class':'pv-entity__secondary-title'})[2].get_text('|',strip=True)
        company2=company2[:company2.rfind('|')]
    except:
        company2=None
    
    df=df.append({'url' : profilelink, 'profile_name' : profile_name,'duration' : duration,'job_title' : job_title,'company1' : company1,'company2' : company2},ignore_index=True)
    
    n=n+1

In [45]:
#df.tail()
df.to_excel('/Users/vpaskannaya/Talent Analytics Live/export_profiles.xls', index = False, header=True)